## Variable Definitions

In [1]:
sns.set()

NameError: name 'sns' is not defined

In [2]:
alpha = 0.05            # default confidence 
upper_cost = 5000000    # <= $5,000,

## DataFrame Definitions

housing_df['zip'] = housing_df['address'].map(lambda x: x.split(',')[-2][-5:])
housing_df['town']= housing_df['address'].map(lambda x: x.split(',')[-3].strip())
housing_df.drop(columns=['address','lat','long'])
kc_only = housing_df.merge(zip_codes['Zip Codes'].astype(str),left_on='zip',right_on='Zip Codes',how='inner')

In [ ]:
display(kc_only.describe(), kc_only.info())

Create homes_df, which only includes houses over 400sqft - which is the point where a house becomes a tiny home.

Additionally, we are only looking at houses that have a bathroom and a bedroom, as well we are cutting off anything above our upper cost range, $5,000,000 in this case.

In [ ]:
homes_df = kc_only[(kc_only['sqft_living'] > 400) &
    (kc_only['bedrooms'] > 0) &
    (kc_only['bathrooms'] > 0) &
    (kc_only['price'] < upper_cost)                        # <= dropping houses valued at and over $5,000,000 - most demographics will not be looking for this.
    ]

Create a dataframe that only has housing data where there are at least 100 houses in a given zip code  
This is to ensure that there is substantial explanatory information when investigating the zip code's effect on pricing later.

In [ ]:
# get zip codes that have at least 100 entries

zip_counts = homes_df['zip'].value_counts().rename('zip_counts')
zips_counted = homes_df.merge(
    zip_counts,
    left_on='zip',
    right_index=True
)
zips_counted.info()
min_zips = zips_counted[zips_counted['zip_counts'] >= 100]
min_zips


In [ ]:
display(homes_df.info(), min_zips.info())

Create a dataframe of zip codes that are sorted by their median price descending, this is useful for finding the highest-priced zip code to use as a reference point when investigating zip code effect on price.

In [ ]:
zip_by_price_median = min_zips[
    ['zip', 'price']].groupby(by='zip').agg(
        {'price': 'median', 'zip': 'size'}
).rename(
    columns={'zip': 'count'}
).reset_index().sort_values('price', ascending=False)
zip_by_price_median

Create an OLS model that describes the zip code's ability to describe price. Output this as a dataframe object that can be called and indexed into.

In [ ]:
y = homes_df['price']
X = sm.add_constant(
    pd.get_dummies(homes_df['zip']).drop(
        # <= selectl the (median) highest-priced zip to drop it
        zip_by_price_median.iloc[0]['zip'],
        axis=1
    ))

zip_model = sm.OLS(y, X).fit().summary2()
print(zip_model)

Create two graphs, one that displays the value of prices by zip code and one that 

In [ ]:
fig, ax = plt.subplots(2, 1, sharex=True, figsize=(20, 10))

y = [zip_by_price_median['price'], zip_by_price_median['count']]
x = zip_by_price_median['zip']

ax[0].bar(x=x, height=y[0])
ax[1].bar(x=x, height=y[1])

ax[0].set(
    title='median house price by zip and count of houses by zip',
    ylabel='Sale Price'
)
ax[1].set(
    ylabel='Number of Houses',
    xlabel='Zip Code',
)

dollar_format = mtick.StrMethodFormatter('${x:,.0f}')
ax[0].yaxis.set_major_formatter(dollar_format)
plt.xticks(rotation=45)
plt.tight_layout()
''